In [3]:
# !pip install segmentation-models-pytorch
# !pip install pytorch_msssim
# !pip install lightning
# !pip install lightning[extra]
# !pip install tensorboard
# !pip install scikit-image

In [4]:
import random
import numpy as np
import os

import torch
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import random_split, Dataset, DataLoader
import cv2

from segmentation_models_pytorch import UnetPlusPlus
import torch.nn.functional as F
import lightning as L
from skimage.metrics import structural_similarity as ssim
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

CFG = {
    'EPOCHS':2,
    'LEARNING_RATE':3e-4,
    # 'BATCH_SIZE':16,
    'BATCH_SIZE':16,
    'SEED':42
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True  # 결정론적 연산 보장
    torch.backends.cudnn.benchmark = False     # 성능 최적화 대신 일관성 우선

seed_everything(CFG['SEED'])  # Seed 고정

def get_input_image(damage_img_path, origin_img_path):
    # OpenCV로 이미지 읽기 (NumPy 배열로 읽음)
    color_image = cv2.imread(origin_img_path)
    gray_image = cv2.imread(damage_img_path, cv2.IMREAD_GRAYSCALE)  # 흑백 이미지로 읽기
    
    # 색상 이미지를 흑백으로 변환 (PIL로 변환 후 NumPy로 변환)
    color_image_gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
    
    # 두 이미지의 차이 계산
    difference = cv2.absdiff(color_image_gray, gray_image)
    
    # 차이 값을 임계값으로 처리하여 이진화 이미지 생성
    _, binary_difference = cv2.threshold(difference, 1, 255, cv2.THRESH_BINARY)

    # 마스크 생성
    mask = binary_difference > 0  # 차이가 있는 부분을 마스크로 설정
    mask = Image.fromarray(mask.astype(np.uint8) * 255)  # 마스크 이미지를 PIL 형식으로 변환

    return {
        'image_gray_masked': Image.fromarray(gray_image),  # 손상된 이미지를 PIL 이미지로 반환
        'mask': transforms.ToTensor()(mask)  # 마스크를 텐서로 변환하여 사용
    }

class CustomDataset(Dataset):
    def __init__(self, damage_dir, origin_dir, transform=None, use_masks=True):
        self.damage_dir = damage_dir
        self.origin_dir = origin_dir
        self.transform = transform
        self.use_masks = use_masks
        self.damage_files = sorted(os.listdir(damage_dir), key=lambda x: x.lower())
        self.origin_files = sorted(os.listdir(origin_dir), key=lambda x: x.lower())

    def __len__(self):
        return len(self.damage_files)

    def __getitem__(self, idx):
        damage_img_name = self.damage_files[idx]
        origin_img_name = self.origin_files[idx]

        damage_img_path = os.path.join(self.damage_dir, damage_img_name)
        origin_img_path = os.path.join(self.origin_dir, origin_img_name)

        damage_img = Image.open(damage_img_path).convert("RGB")
        origin_img = Image.open(origin_img_path).convert("RGB")

        if self.use_masks:
            tmp_damage_img = damage_img.convert("L")
            input_data = get_input_image(damage_img_path, origin_img_path)
            mask = transforms.ToTensor()(input_data['mask'])
        else:
            mask = torch.zeros((1, damage_img.size[1], damage_img.size[0]))

        if self.transform:
            damage_img = self.transform(damage_img)
            origin_img = self.transform(origin_img)

        return {'A': damage_img, 'B': origin_img, 'mask': mask}



def get_histogram_similarity(true_np, pred_np, color_space=cv2.COLOR_RGB2HSV):
    # BGR 이미지를 HSV로 변환
    true_hsv = cv2.cvtColor(true_np.astype(np.uint8), color_space)
    pred_hsv = cv2.cvtColor(pred_np.astype(np.uint8), color_space)
    
    # 히스토그램 계산 및 비교
    hist_true = cv2.calcHist([true_hsv], [0], None, [180], [0, 180])
    hist_pred = cv2.calcHist([pred_hsv], [0], None, [180], [0, 180])
    
    hist_true = cv2.normalize(hist_true, hist_true).flatten()
    hist_pred = cv2.normalize(hist_pred, hist_pred).flatten()
    
    similarity = cv2.compareHist(hist_true, hist_pred, cv2.HISTCMP_CORREL)
    return similarity

def get_masked_ssim_score(true_np, pred_np, mask_np):
    # true_np: (height, width, channels)
    # pred_np: (height, width, channels)
    # mask_np: (height, width) or (1, height, width)
    
    # mask_np 차원 변환
    if mask_np.ndim == 3 and mask_np.shape[0] == 1:
        mask_np = mask_np.squeeze(0)  # (1, height, width) -> (height, width)
    elif mask_np.ndim == 3 and mask_np.shape[-1] == 1:
        mask_np = mask_np.squeeze(-1)  # (height, width, 1) -> (height, width)
    
    # true_np와 mask_np의 크기가 맞지 않으면 리사이즈
    if true_np.shape[:2] != mask_np.shape:
        mask_np = cv2.resize(mask_np, (true_np.shape[1], true_np.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    # 마스크가 적용된 부분만 추출
    true_masked = true_np[mask_np > 0]
    pred_masked = pred_np[mask_np > 0]

    if true_masked.size == 0 or pred_masked.size == 0:
        return 0  # 마스크된 부분이 없으면 SSIM을 0으로 처리

    # SSIM 계산
    ssim_value = ssim(
        true_masked, pred_masked, data_range=pred_masked.max() - pred_masked.min(), channel_axis=-1
    )
    return ssim_value


def ssim_score(true, pred):
    true_np = true.permute(0, 2, 3, 1).cpu().numpy()
    pred_np = pred.permute(0, 2, 3, 1).cpu().numpy()

    print(f"true_np shape: {true_np.shape}")
    print(f"pred_np shape: {pred_np.shape}")
    print(f"mask_np shape: {mask_np.shape}")

    scores = [
        ssim(t, p, channel_axis=-1, data_range=p.max() - p.min())
        for t, p in zip(true_np, pred_np)
    ]
    return np.mean(scores)

def get_ssim_score(true_np, pred_np):
    """
    두 이미지를 비교하여 SSIM(Structural Similarity Index Measure)을 계산합니다.
    NaN 값을 방지하기 위해 추가적인 조건을 추가합니다.

    Args:
        true_np (numpy.ndarray): Ground truth 이미지 (H, W, C 형태)
        pred_np (numpy.ndarray): 예측 이미지 (H, W, C 형태)

    Returns:
        float: SSIM 값 (NaN 발생 시 0으로 설정)
    """
    ssim_value = ssim(
        true_np, pred_np, channel_axis=-1, data_range=pred_np.max() - pred_np.min()
    )
    if np.isnan(ssim_value):
        ssim_value = 0  # NaN 발생 시 SSIM 값을 0으로 설정
    return ssim_value

# LitIRModel 클래스 정의
class LitIRModel(L.LightningModule):
    def __init__(self, model_1, model_2, image_mean=0.5, image_std=0.5):
        super().__init__()
        self.model_1 = model_1
        self.model_2 = model_2
        self.image_mean = image_mean
        self.image_std = image_std
        self.validation_outputs = []

        self.val_score = 0

    def forward(self, images_gray_masked):
        images_gray_restored = self.model_1(images_gray_masked) + images_gray_masked
        images_restored = self.model_2(images_gray_restored)
        return images_gray_restored, images_restored

    def unnormalize(self, output, round=False):
        image_restored = ((output * self.image_std + self.image_mean) * 255).clamp(0, 255)
        if round:
            image_restored = torch.round(image_restored)
        return image_restored

    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=1e-5)
        return opt

    def training_step(self, batch, batch_idx):
        images_gray_masked = batch['A']  # 손상된 이미지 정답: mask가 채워진 흑백사진
        images_gt = batch['B']  # Ground Truth 이미지
        masks = batch['masks']  # 마스크 (현재 사용 여부에 따라 다름)

        # 모델에 입력
        images_gray_restored, images_restored = self(images_gray_masked)

        # 손실 계산
        loss_pixel_gray = ( # F.l1_loss(정답:구멍채워진흑백, 즉 컬러를 흑백으로 바꾸면됨, 예측:구멍채우려고 노력하는model1, reductio
            F.l1_loss(images_gray_masked, images_gray_restored, reduction='mean') * 0.5 +
            F.mse_loss(images_gray_masked, images_gray_restored, reduction='mean') * 0.5
        )
        loss_pixel = (
            F.l1_loss(images_gt, images_restored, reduction='mean') * 0.5 +
            F.mse_loss(images_gt, images_restored, reduction='mean') * 0.5
        )
        loss = loss_pixel_gray * 0.5 + loss_pixel * 0.5

        # 로깅 (Batch와 손실 값 출력)
        print(f"Batch {batch_idx}, Loss: {loss.item()}")
        
        # 로그 기록
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Extract data from batch
        images_gray_masked = batch['A']
        images_gt = batch['B']
        masks = batch['masks']

        # Forward pass
        images_gray_restored, images_restored = self(images_gray_masked)

        # Ground Truth와 복원된 이미지 크기 맞추기
        if images_restored.shape != images_gt.shape:
            images_restored = torch.nn.functional.interpolate(
                images_restored, size=images_gt.shape[-2:], mode="bilinear", align_corners=False
            )

        # NumPy 변환
        images_restored_np = images_restored.detach().cpu().permute(0, 2, 3, 1).float().numpy().astype(np.uint8)
        images_gt_np = images_gt.detach().cpu().permute(0, 2, 3, 1).float().numpy().astype(np.uint8)
        masks_np = masks.detach().cpu().numpy()

        # Metric 계산
        total_ssim_score = 0
        masked_ssim_score = 0
        hist_sim_score = 0
        for image_gt_np, image_restored_np, mask_np in zip(images_gt_np, images_restored_np, masks_np):
            total_ssim_score += get_ssim_score(image_gt_np, image_restored_np) / len(images_gt)
            masked_ssim_score += get_masked_ssim_score(image_gt_np, image_restored_np, mask_np) / len(images_gt)
            hist_sim_score += get_histogram_similarity(image_gt_np, image_restored_np) / len(images_gt)

        # 최종 점수
        score = total_ssim_score * 0.2 + masked_ssim_score * 0.4 + hist_sim_score * 0.4
        val_score += score

        # 배치별 검증 결과 출력
        print(f"Validation Batch {batch_idx}, SSIM Score: {total_ssim_score:.4f}")

        # 로깅
        self.log("val_score", score, on_step=False, on_epoch=True)
        self.log("val_total_ssim_score", total_ssim_score, on_step=False, on_epoch=True)
        self.log("val_masked_ssim_score", masked_ssim_score, on_step=False, on_epoch=True)
        self.log("val_hist_sim_score", hist_sim_score, on_step=False, on_epoch=True)

    def on_validation_epoch_end(self):
        total_ssim_score = 0
        masked_ssim_score = 0
        hist_sim_score = 0

        for output in self.validation_outputs:
            images_gt = output["images_gt"]
            images_restored = output["images_restored"]
            masks = output["masks"]

            masks_np = masks.detach().cpu().numpy()
            images_gt_np = images_gt.detach().cpu().permute(0, 2, 3, 1).float().numpy().astype(np.uint8)
            images_restored_np = images_restored.detach().cpu().permute(0, 2, 3, 1).float().numpy().astype(np.uint8)

            for image_gt_np, image_restored_np, mask_np in zip(images_gt_np, images_restored_np, masks_np):
                total_ssim_score += get_ssim_score(image_gt_np, image_restored_np) / len(images_gt_np)
                masked_ssim_score += get_masked_ssim_score(image_gt_np, image_restored_np, mask_np) / len(images_gt_np)
                hist_sim_score += get_histogram_similarity(image_gt_np, image_restored_np, cv2.COLOR_RGB2HSV) / len(images_gt_np)

        score = total_ssim_score * 0.2 + masked_ssim_score * 0.4 + hist_sim_score * 0.4

        self.log(f"val_score", score, on_step=False, on_epoch=True)
        self.log(f"val_total_ssim_score", total_ssim_score, on_step=False, on_epoch=True)
        self.log(f"val_masked_ssim_score", masked_ssim_score, on_step=False, on_epoch=True)
        self.log(f"val_hist_sim_score", hist_sim_score, on_step=False, on_epoch=True)

        self.validation_outputs = []

# 모델 초기화
# 첫 번째 모델: Gray Mask Restoration
model_1 = UnetPlusPlus(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
)

# 두 번째 모델: Gray → Color
model_2 = UnetPlusPlus(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=3,
    classes=3
)

# LitIRModel 초기화
lit_ir_model = LitIRModel(model_1=model_1, model_2=model_2)



# 경로 설정
origin_dir = 'data/train_gt'
damage_dir = 'data/train_input'
test_dir = 'data/test_input'

# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 모든 이미지를 256x256으로 리사이즈
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 데이터셋 생성
dataset = CustomDataset(damage_dir=damage_dir, origin_dir=origin_dir, transform=transform)

# 데이터셋 분할
validation_ratio = 0.2
train_size = int((1 - validation_ratio) * len(dataset))
val_size = len(dataset) - train_size
training_dataset, validation_dataset = random_split(dataset, [train_size, val_size])

class CollateFn:
    def __init__(self, mode='train'):
        self.mode = mode

    def __call__(self, batch):
        A = torch.stack([item['A'] for item in batch])
        B = torch.stack([item['B'] for item in batch])
        masks = torch.stack([item['mask'] for item in batch]) if 'mask' in batch[0] else torch.zeros_like(A)

        if self.mode in ['train', 'valid']:
            return {'A': A, 'B': B, 'masks': masks}
        elif self.mode == 'test':
            return {'A': A}

# CollateFn 정의
train_collate_fn = CollateFn(mode='train')
validation_collate_fn = CollateFn(mode='valid')

# DataLoader 설정
train_dataloader = DataLoader(
    training_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    collate_fn=train_collate_fn
)

validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=False,
    num_workers=8,
    collate_fn=validation_collate_fn
)

# 모델 저장 디렉토리 생성
model_save_dir = "./saved_models"
os.makedirs(model_save_dir, exist_ok=True)

# Trainer 설정 및 학습 시작
trainer = L.Trainer(
    max_epochs=CFG['EPOCHS'],
    precision=16,
    accelerator='gpu',
    devices=1,
    log_every_n_steps=10,
    callbacks=[
        ModelCheckpoint(
            monitor='val_score',
            mode='max',
            save_top_k=1,
            dirpath=model_save_dir,
            filename='best_model-{epoch:02d}-{val_score:.4f}'
        ),
        EarlyStopping(monitor='val_score', mode='max', patience=3)
    ]
)

# 학습 시작
trainer.fit(lit_ir_model, train_dataloader, validation_dataloader)

ImportError: cannot import name 'log_trace_structured_event' from 'torch._utils_internal' (c:\Users\zqrc0\anaconda3\envs\tensorflow-env\lib\site-packages\torch\_utils_internal.py)